# training-data-analyst/courses/machine_learning/deepdive2/introduction_to_tensorflow

### Loss function and gradients

**tf.GradientTape** 참고: [Gradient Tape을 이용한 자동 미분과 오차역전파](https://rfriend.tistory.com/m/556?category=711317)

In [1]:
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)

def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:  # forward
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])  # backward

In [ ]:
def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True)
    return dataset

In [ ]:
def create_dataset(pattern, batch_size=1, mode='eval'):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels).cache()

    if mode == 'train':
        dataset = dataset.shuffle(1000).repeat(3)

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(buffer_size=1)
    
    return dataset

In [114]:
# Build a keras DNN model using Sequential API
# TODO 2a
model = tf.keras.Sequential([
    DenseFeatures(feature_columns=feature_columns),
    Dense(units=32, activation="relu", name="h1"),
    Dense(units=8, activation="relu", name="h2"),
    Dense(units=1, activation="linear", name="output")

])

In [115]:
# TODO 2b
# Create a custom evalution metric
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true-y_pred)**2))


# Compile the keras model
# TODO -- Your code here.
model.compile(optimizer='rmsprop', loss='mse', metrics=['mse', rmse])

In [126]:
%%bash

# TODO 5a

PROJECT="qwiklabs-gcp-00-0db9b1bc58c6"
BUCKET=${PROJECT}
REGION=us-east1
MODEL_NAME=taxifare
VERSION_NAME=dnn1

## Create GCS bucket if it doesn't exist already...
exists=$(gsutil ls -d | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket exists, let's not recreate it."
else
    echo "Creating a new GCS bucket."
    gsutil mb -l ${REGION} gs://${BUCKET}
    echo "Here are your current buckets:"
    gsutil ls
fi

if [[ $(gcloud ai-platform models list --format='value(name)' --region=$REGION | grep $MODEL_NAME) ]]; then
    echo "$MODEL_NAME already exists"
else
    echo "Creating $MODEL_NAME"
    gcloud ai-platform models create --region=$REGION $MODEL_NAME
fi

if [[ $(gcloud ai-platform versions list --model $MODEL_NAME --region=$REGION --format='value(name)' | grep $VERSION_NAME) ]]; then
    echo "Deleting already existing $MODEL_NAME:$VERSION_NAME ... "
    echo yes | gcloud ai-platform versions delete --model=$MODEL_NAME $VERSION_NAME --region=$REGION
    echo "Please run this cell again if you don't see a Creating message ... "
    sleep 2
fi

echo "Creating $MODEL_NAME:$VERSION_NAME"
gcloud ai-platform versions create \
    --model=taxifare $VERSION_NAME \
    --framework=tensorflow \
    --python-version=3.7 \
    --runtime-version=2.1 \
    --origin=$EXPORT_PATH \
    --staging-bucket=gs://$BUCKET \
    --region=$REGION

Bucket exists, let's not recreate it.
taxifare already exists
Deleting already existing taxifare:dnn1 ... 
Please run this cell again if you don't see a Creating message ... 
Creating taxifare:dnn1


Using endpoint [https://us-east1-ml.googleapis.com/]
Using endpoint [https://us-east1-ml.googleapis.com/]
Using endpoint [https://us-east1-ml.googleapis.com/]
This will delete version [dnn1]...

Do you want to continue (Y/n)?  
Deleting version [dnn1]......
..................................................................................done.
Using endpoint [https://us-east1-ml.googleapis.com/]
Creating version (this might take a few minutes)......
....................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................done.
